In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from datetime import date, datetime, timedelta, time
import pandas as pd
import seaborn
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import IPython.html.widgets as widgets

In [ ]:
import scipy
from scipy.stats import norm
import scipy.optimize as op
import emcee

In [ ]:
seaborn.set_context('talk')

In [ ]:
from simtransient.models.supernova import Sn1aOpticalEnsemble

In [ ]:
ens = Sn1aOpticalEnsemble()

In [ ]:
ens.gauss_pars

In [ ]:
ens.gauss_correlations

In [ ]:
print "det:", np.linalg.det(ens.gauss_cov)
ens.gauss_cov

In [ ]:
from scipy.stats import multivariate_normal
gauss_hypers_rv = multivariate_normal(mean=ens.gauss_pars.loc['mu'], cov=ens.gauss_cov)

In [ ]:
true_params = pd.Series(index=ens.gauss_pars.keys(), 
                        data=gauss_hypers_rv.rvs(), 
                        name="true_params")
true_params['t0']=0
true_params

In [ ]:
dict(true_params)

In [ ]:
true_curve = ens.get_curve(**true_params)
true_curve

In [ ]:
n_samples=5000
sample = gauss_hypers_rv.rvs(n_samples)
import triangle

_=triangle.corner(sample,
                 labels=ens.gauss_pars.keys(),
                 quantiles=[0.05, 0.5, 0.95],
                 truths=true_params)

In [ ]:
true_params

In [ ]:

def plt_traces(start,ntrace):
#     ntrace=int(ntrace)
    stop = min(start+ntrace, n_samples)
    plt_samples = sample[start:stop]
    lcs = []
    t=np.linspace(-20, 80,1000)
    for pltpars in plt_samples:
        lcs.append(ens.evaluate(t,*pltpars,t0=0))
    lcs=np.array(lcs)
    seaborn.tsplot( lcs,  t, err_style="unit_traces", ls='')

widgets.interact_manual(plt_traces, 
         start=(0,n_samples-1,1),
         ntrace=(0,150),
        )


In [ ]:

@widgets.interact(a=(0.9,1.4,0.1), rise_tau=(2.5,3.5,0.1), decay_tau=(12,20,0.1), t0=(0,10,2))
def plot_sne(a,rise_tau,decay_tau,t0):
    curve = ens.get_curve(a=a,rise_tau=rise_tau,decay_tau=decay_tau,t0=t0)
    t=np.linspace(-rise_tau*5, decay_tau*5,1000)
#     flux = sn_instance(t)
    flux = curve(t)
    plt.plot(t,flux)
    plt.ylim(0,2)
    plt.xlim(-15,80)
    if False:
        triangle.corner(sample,
                     labels=ens.gpars.keys(),
                     truths=(a,rise_tau,decay_tau)
                       )